# Quickstart VERONA tutorial

In this section, we describe the main components of the VERONA API required to perform full-set robustness evaluation experiments.  
We illustrate these components by providing examples of concrete classes we readily provide in VERONA.


## Datasets
The first step is to import a dataset. VERONA supports two main options:  
(i) loading a custom dataset (see the README for details on the required file structure), or  
(ii) reusing existing datasets available through torchvision.  
We illustrate the first with the MNIST dataset examples available in examples/example_experiment.  

In [2]:
from pathlib import Path
from ada_verona.database.image_file_dataset import ImageFileDataset


# Import a custom dataset
# The required file structure is described in the README
dataset = ImageFileDataset(
    image_folder= Path("../example_experiment/data/images"), 
    label_file=Path("../example_experiment/data/image_labels.csv"), 
    preprocessing=None
)



ModuleNotFoundError: No module named 'ada_verona.database.image_file_dataset'

VERONA also includes dataset samplers.  
These allow you to restrict verification to specific subsets of data, for example, only verifying correctly classified instances or several samples from each class. 

In [ ]:
from ada_verona.dataset_sampler.dataset_sampler import DatasetSampler
from ada_verona.dataset_sampler.predictions_based_sampler import PredictionsBasedSampler

# A sampler based on model predictions  
# sample_correct_predictions = True: only include correctly classified instances  
# sample_correct_predictions = False : only include misclassified instances  
sampler = PredictionsBasedSampler(
    sample_correct_predictions=True
)

## Robustness Evaluation Algorithms
The next step is to determine which algorithm will be used to evaluate robustness.  
VERONA supports complete verifiers as well as commonly used adversarial attacks.  
We illustrate the use of a readily available attack, called PGD. 

In [ ]:

from ada_verona.verification_module.attacks.pgd_attack import PGDAttack
from ada_verona.verification_module.attack_estimation_module import AttackEstimationModule

# Define a PGD attack instance
attack = PGDAttack(
    number_iterations=100,
    stepsize=0.2,
    randomise=True
)

verifier = AttackEstimationModule(attack=attack)


## Estimators 

The $\varepsilon$ values at which robustness is evaluated can be chosen in different ways.  
Currently, two estimators are supported:  
(i) an iterative search, which evaluates all user-specified $\varepsilon$ values, and  
(ii) a binary search, which efficiently identifies adjacent unsat and sat values to approximate $\tilde{\varepsilon}^*$.  

We illustrate the binary search.

In [ ]:
from ada_verona.epsilon_value_estimator.epsilon_value_estimator import EpsilonValueEstimator
from ada_verona.epsilon_value_estimator.iterative_value_estimator import IterativeValueEstimator
from ada_verona.epsilon_value_estimator.binary_search_epsilon_value_estimator import BinarySearchEpsilonValueEstimator

# Select the epsilon estimator
epsilons = [0.1, 0.2, 0.3, 0.4, 0.5]


# Binary search
# Finds adjacent unsat and sat values in epsilons
estimator = BinarySearchEpsilonValueEstimator(
    epsilon_value_list=epsilons,
    verifier=verifier
)


## Property generators

A property generator specifies the property to be verified.  
VERONA currently provides support for both targeted robustness (One2One) and untargeted local robustness (One2Any), expressed as vnnlib properties.  
Adding propertygenerator is possible, but the verifier needs to be able to read and handle the property.


In [ ]:

from ada_verona.verification_module.property_generator.one2any_property_generator import One2AnyPropertyGenerator

# Select the property type
# VERONA currently provides two common property generators (compatible with most verifiers)

# Untargeted local robustness (most common)
# Creates vnnlib properties for verifying robustness against any class
property = One2AnyPropertyGenerator(
    number_classes=10,
    data_lb=0,
    data_ub=1
)


Finally, an experiment can be created, configured, and executed using the experiment repository.  
This repository manages experiment metadata, sampled data points, verification contexts, and results. 

In [ ]:
from ada_verona.database.experiment_repository import ExperimentRepository

experiment_name = "test_experiments"
experiment_repository_path=Path("../example_experiment")
network_folder=Path("../example_experiment/data/networks")


# The network folder should be a directory containing networks.
experiment_repository = ExperimentRepository(
    base_path= experiment_repository_path,
    network_folder=network_folder
)

# Initialise a new experiment
experiment_repository.initialize_new_experiment(experiment_name)

# Save the experiment configuration
experiment_repository.save_configuration(
    dict(
        experiment_name=experiment_name,
        experiment_repository_path=experiment_repository_path,
        network_folder=str(network_folder),
        dataset=str(dataset),
        timeout=360,
        epsilon_list=[str(x) for x in epsilons],
    )
)

# Run verification for each network
for network in network_list:
    sampled_data = dataset_sampler.sample(network, dataset)
    for data_point in sampled_data:
        verification_context = experiment_repository.create_verification_context(
            network,
            data_point,
            property_generator
        )
    
        # At this point you can either:
        # 1. Save the VerificationContext to execute later on (e.g., via SLURM), or
        # 2. Directly execute, as illustrated here:
        epsilon_value_result = epsilon_value_estimator.compute_epsilon_value(
            verification_context
        )
    
        experiment_repository.save_result(epsilon_value_result)